# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
data1= data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'})
display(data1)
data1.columns

Quantity
CustomerID ProductName                                
33         Apricots - Dried                          1
           Assorted Desserts                         1
           Bandage - Flexible Neon                   1
           Bar Mix - Pina Colada, 355 Ml             1
           Beans - Kidney, Canned                    1
           Beef - Chuck, Boneless                    1
           Beef - Prime Rib Aaa                      1
           Beer - Original Organic Lager             1
           Beer - Rickards Red                       1
           Black Currants                            1
           Blackberries                              1
           Bread - Italian Roll With Herbs           1
           Bread Foccacia Whole                      1
           Cake - Box Window 10x10x2.5               1
           Cassis                                    1
           Cheese - Cottage Cheese                   1
           Crab - Imitation Flakes                   1
           Fondant - Icing                           1
           French Pastry - Mini Chocolate            1
           General Purpose Trigger                   1
           Grapes - Red                              1
           Grouper - Fresh                           2
           Hersey Shakes                             1
           Ice Cream Bar - Drumstick                 1
           Lamb - Pieces, Diced                      1
           Lettuce - Spring Mix                      2
           Mangoes                                   1
           Muffin Batt - Choc Chk                    1
           Onions - Vidalia                          1
           Pasta - Cheese / Spinach Bauletti         1
...                                                ...
98200      Onion Powder                             25
           Orange - Canned, Mandarin                25
           Pasta - Detalini, White, Fresh           25
           Puree - Passion Fruit                    25
           Raspberries - Fresh                      25
           Rice - Long Grain                        25
           Rosemary - Dry                           25
           Rosemary - Primerba, Paste               50
           Rum - Coconut, Malibu                    25
           Sage - Ground                            25
           Sausage - Liver                          25
           Sea Bass - Whole                         50
           Seedlings - Mix, Organic                 25
           Sole - Dover, Whole, Fresh               25
           Soup - Campbells, Lentil                 50
           Soup Campbells - Italian Wedding         25
           Soup Knorr Chili With Beans              25
           Soupfoamcont12oz 112con                  25
           Spinach - Baby                           25
           Table Cloth 54x72 White                  25
           Tea - English Breakfast                  25
           Tilapia - Fillets                        25
           Tomatoes Tear Drop                       25
           Truffle Cups - Brown                     25
           Vaccum Bag 10x13                         25
           Vol Au Vents                             50
           Wasabi Powder                            25
           Wine - Fume Blanc Fetzer                 25
           Wine - Hardys Bankside Shiraz            25
           Yogurt - French Vanilla                  25

[63628 rows x 1 columns]

Index(['Quantity'], dtype='object')

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
data2= pd.pivot_table(data, index=['ProductName'], values='Quantity', columns=['CustomerID'],fill_value=0)
data2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
from scipy.spatial.distance import pdist, squareform


In [7]:

euclideando = pd.DataFrame(1/(1 + squareform(pdist(data2.T, 'euclidean'))), index=data2.columns, columns=data2.columns)
euclideando.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
best_costumers=euclideando[264].sort_values(ascending=False)[1:6].index.tolist()
print(best_costumers)

[33, 2915, 1920, 1072, 3472]


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [13]:
data1=data1.reset_index()
data1.head()
data3=data1.loc[data1['CustomerID'].isin(best_costumers)]
display(data3)

,index,CustomerID,ProductName,Quantity
0,0,33,Apricots - Dried,1
1,1,33,Assorted Desserts,1
2,2,33,Bandage - Flexible Neon,1
3,3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,4,33,"Beans - Kidney, Canned",1
5,5,33,"Beef - Chuck, Boneless",1
6,6,33,Beef - Prime Rib Aaa,1
7,7,33,Beer - Original Organic Lager,1
8,8,33,Beer - Rickards Red,1
9,9,33,Black Currants,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [17]:
data5= data3.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).sort_values(by=['Quantity'], ascending=False)
data5


Quantity
CustomerID ProductName                               
1920       Pork - Kidney                            3
2915       Cheese - Parmesan Grated                 3
1920       Lime Cordial - Roses                     2
2915       Sugar - Fine                             2
3472       Pasta - Angel Hair                       2
           Juice - Lime                             2
33         Salsify, Organic                         2
           Veal - Inside, Choice                    2
3472       Beets - Mini Golden                      2
2915       Veal - Brisket, Provimi,bnls             2
           V8 - Berry Blend                         2
           Turkey - Oven Roast Breast               2
1072       Muffin - Carrot Individual Wrap          2
3472       Pepper - White, Ground                   2
2915       Salmon Steak - Cohoe 8 Oz                2
           Quiche Assorted                          2
1072       Wine - Chardonnay South                  2
2915       Crush - Cream Soda                       2
1920       Cheese - Boursin, Garlic / Herbs         2
           Vanilla Beans                            2
           Ice Cream Bar - Hageen Daz To            2
33         Lettuce - Spring Mix                     2
2915       Tia Maria                                2
3472       Soup - Campbells Tomato Ravioli          2
33         Grouper - Fresh                          2
3472       Tomato - Tricolor Cherry                 2
           Sage - Ground                            2
2915       Hersey Shakes                            1
           Cod - Black Whole Fillet                 1
           Coffee - Hazelnut Cream                  1
...                                               ...
1072       Pasta - Detalini, White, Fresh           1
           Crush - Cream Soda                       1
           Fish - Scallops, Cold Smoked             1
           Foam Dinner Plate                        1
           Fondant - Icing                          1
           Juice - Orange                           1
           Knife Plastic - White                    1
           Lentils - Red, Dry                       1
           Longos - Grilled Chicken With            1
           Macaroons - Two Bite Choc                1
           Mushroom - Porcini, Dry                  1
           Oil - Shortening - All - Purpose         1
           Olive - Spread Tapenade                  1
           Otomegusa Dashi Konbu                    1
           Pasta - Penne, Rigate, Dry               1
           Whmis - Spray Bottle Trigger             1
           Quiche Assorted                          1
           Rice - Jasmine Sented                    1
           Sausage - Liver                          1
           Scallop - St. Jaques                     1
           Sherry - Dry                             1
           Sobe - Tropical Energy                   1
           Sole - Dover, Whole, Fresh               1
           Soup - Campbells Bean Medley             1
           Soup Campbells - Italian Wedding         1
           Spinach - Baby                           1
           Sprouts - Baby Pea Tendrils              1
           Table Cloth 54x72 White                  1
           Water, Tap                               1
3472       Zucchini - Yellow                        1

[267 rows x 1 columns]

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [32]:
data_merge = pd.merge(data5, data2,  on='ProductName', how='inner').sort_values(by=['Quantity'], ascending=False)


In [29]:
result_264 = data_merge[data_merge[264]==0].sort
result_264.head()

,Quantity,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Pork - Kidney,3,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,0,0
Cheese - Parmesan Grated,3,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Cheese - Parmesan Grated,1,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,0,0
Lime Cordial - Roses,2,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sugar - Fine,2,0,0,0,0,0,1,0,0,1,...,0,0,0,25,0,0,25,0,0,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.